In [1]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split 
from sklearn.metrics import r2_score, mean_squared_error as mse
from sklearn.preprocessing import MinMaxScaler

In [2]:
df = pd.read_csv("C:/Training/Academy/Statistics (Python)/Cases/Real Estate/Housing.csv")
dum_df = pd.get_dummies(df,drop_first=True)
dum_df = dum_df.astype(float)
dum_df.head()

,price,lotsize,bedrooms,bathrms,stories,garagepl,driveway_yes,recroom_yes,fullbase_yes,gashw_yes,airco_yes,prefarea_yes
0,42000.0,5850.0,3.0,1.0,2.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
1,38500.0,4000.0,2.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,49500.0,3060.0,3.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,60500.0,6650.0,3.0,1.0,2.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
4,61000.0,6360.0,2.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [3]:
dum_df.shape

(546, 12)

In [4]:
X = dum_df.iloc[:,1:]
y = dum_df['price']
scaler = MinMaxScaler()
scalerY = MinMaxScaler()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, 
                                                    random_state=23)
y_train = scalerY.fit_transform(y_train.values.reshape(-1, 1))
X_train = scaler.fit_transform(X_train)
#y_test = scalerY.transform(y_test.values.reshape(-1, 1))
X_test = scaler.transform(X_test)

In [5]:
X_torch = torch.from_numpy(X_train)
y_torch = torch.from_numpy(y_train)
print(X_torch.size())
print(y_torch.size())

torch.Size([382, 11])
torch.Size([382, 1])


In [6]:
torch.manual_seed(23)
# Create a model
model = nn.Sequential(nn.Linear(in_features=X_train.shape[1], out_features=8),
                      nn.ReLU(),
                      nn.Linear(8,6),
                      nn.ReLU(),
                      nn.Linear(6,4),
                      nn.ReLU(),
                      nn.Linear(4, 3),
                      nn.ReLU(),
                      nn.Linear(3,1),
                      nn.ReLU())

In [7]:
criterion = torch.nn.MSELoss()
# Construct the optimizer (Stochastic Gradient Descent in this case)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)

Initail Weights

In [8]:
model[0].weight

Parameter containing:
tensor([[-0.0433, -0.1273, -0.0468, -0.0862,  0.2760, -0.2352, -0.1246,  0.2536,
          0.0528, -0.2232,  0.1042],
        [-0.2395,  0.1735,  0.0325, -0.0209, -0.1622, -0.1680, -0.0996, -0.0277,
         -0.1496,  0.0790, -0.1986],
        [-0.1132, -0.1824,  0.0281, -0.2887,  0.2442,  0.2077,  0.2611, -0.1236,
         -0.0137, -0.0128, -0.0941],
        [ 0.1044,  0.0961, -0.1882, -0.0274, -0.2383,  0.1274,  0.1634,  0.0310,
         -0.2528, -0.1189,  0.1770],
        [-0.1313,  0.0414, -0.2199, -0.0468, -0.1842, -0.2538, -0.1663, -0.1138,
          0.2585,  0.2574,  0.1214],
        [ 0.1681,  0.0995, -0.1351,  0.1339, -0.0150, -0.2048, -0.0473,  0.1629,
         -0.0395,  0.0133,  0.0892],
        [ 0.1051, -0.2302, -0.1703,  0.1509,  0.2103, -0.0927,  0.1740,  0.0780,
          0.2308,  0.1270, -0.0150],
        [ 0.2325, -0.1317, -0.0337,  0.0919,  0.1691, -0.2744, -0.0111,  0.2391,
          0.2764,  0.1646,  0.1470]], requires_grad=True)

In [9]:
model

Sequential(
  (0): Linear(in_features=11, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=6, bias=True)
  (3): ReLU()
  (4): Linear(in_features=6, out_features=4, bias=True)
  (5): ReLU()
  (6): Linear(in_features=4, out_features=3, bias=True)
  (7): ReLU()
  (8): Linear(in_features=3, out_features=1, bias=True)
  (9): ReLU()
)

In [10]:
y_pred = model(X_torch.float())
#y_torch = y_torch.unsqueeze(1)
print(y_torch.shape)
print(y_pred.shape)

torch.Size([382, 1])
torch.Size([382, 1])


In [11]:
# Gradient Descent

for epoch in np.arange(0,1000):
   # Forward pass: Compute predicted y by passing x to the model
   y_pred = model(X_torch.float())

   # Compute and print loss
   loss = criterion(y_pred, y_torch.float())
   #print('epoch: ', epoch+1,' loss: ', loss.item())

   # Zero gradients, perform a backward pass, and update the weights.
   optimizer.zero_grad()

   # perform a backward pass (backpropagation)
   loss.backward()

   # Update the parameters
   optimizer.step()
   if epoch % 100 == 0:
       print('epoch: ', epoch+1,' loss: ', loss.item())

epoch:  1  loss:  0.14013861119747162
epoch:  101  loss:  0.050157006829977036
epoch:  201  loss:  0.03031545877456665
epoch:  301  loss:  0.022157352417707443
epoch:  401  loss:  0.014629005454480648
epoch:  501  loss:  0.010371306911110878
epoch:  601  loss:  0.008616790175437927
epoch:  701  loss:  0.007804907392710447
epoch:  801  loss:  0.007258238736540079
epoch:  901  loss:  0.006876250263303518


In [12]:
#model = model.eval()
X_torch_test = torch.from_numpy(X_test)
y_pred = model(X_torch_test.float())
y_pred = y_pred.detach().numpy()
y_pred = y_pred.reshape(y_test.shape[0],)
y_pred[:5]

array([0.45926726, 0.1443369 , 0.20523548, 0.14433706, 0.1933908 ],
      dtype=float32)

In [13]:
y_pred = y_pred.reshape(-1,1)
y_pred_orig = scalerY.inverse_transform(y_pred)
y_pred_orig[:5]

array([[100779.09 ],
       [ 48815.586],
       [ 58863.855],
       [ 48815.617],
       [ 56909.48 ]], dtype=float32)

In [14]:
y_test.iloc[:5]

231    87000.0
155    60000.0
424    65500.0
268    54000.0
19     45000.0
Name: price, dtype: float64

### Test Set R2 Score

In [15]:
print(r2_score(y_test,y_pred_orig))

0.5625960372869746


### Test Set MSE

In [16]:
mse(y_test,y_pred_orig)

282051566.03874516